In [11]:
import copy
import os
import argparse
import torch
import numpy as np
import random

%load_ext autoreload
%autoreload 2
from erasure.utils.config.local_ctx import Local
from erasure.utils.config.global_ctx import Global 
from erasure.core.factory_base import ConfigurableFactory
from erasure.data.datasets.DatasetManager import DatasetManager

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
config_file = os.path.join("configs", "gnn", "ESOL.jsonc")
#config_file = os.path.join("configs", "example.jsonc")

global_ctx = Global(config_file)
global_ctx.factory = ConfigurableFactory(global_ctx)

#Create Dataset
data_manager = global_ctx.factory.get_object( Local( global_ctx.config.data ))
global_ctx.dataset = data_manager

2025-02-17 18:47:47,352712438 | INFO | 739601 - Creating Global Context for: configs/gnn/ESOL.jsonc


2025-02-17 18:47:47,352712485 | INFO | 739601 - Setting seeds to: 2
2025-02-17 18:47:47,352712516 | INFO | 739601 - Caching System: False.
2025-02-17 18:47:47,352712641 | INFO | 739601 - Instantiating: torch_geometric.datasets.MoleculeNet
2025-02-17 18:47:47,352712647 | INFO | 739601 - Created Configurable: erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource
2025-02-17 18:47:47,352712648 | INFO | 739601 - {'class': 'erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource', 'parameters': {'datasource': {'class': 'torch_geometric.datasets.MoleculeNet', 'parameters': {'root': 'resources/data', 'name': 'ESOL'}, 'preprocess': []}}}
2025-02-17 18:47:47,352713058 | INFO | 739601 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
2025-02-17 18:47:47,352713105 | INFO | 739601 - ['all', 'all_shuffled', '-']
2025-02-17 18:47:47,352713106 | INFO | 739601 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
20

In [3]:
data_manager.n_classes

2

In [13]:
from erasure.utils.config.global_ctx import bcolors

#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

2025-02-17 11:08:07,325132501 | INFO | 569228 - Instantiating: erasure.model.gcn.DownstreamGCN
2025-02-17 11:08:07,325132599 | INFO | 569228 - Instantiating: torch.optim.RMSprop
2025-02-17 11:08:07,325132600 | INFO | 569228 - Instantiating: torch.nn.CrossEntropyLoss
2025-02-17 11:08:07,325132602 | INFO | 569228 - TRAINING WITH 1210 samples
2025-02-17 11:08:07,325132833 | INFO | 569228 - epoch = 0 ---> loss = 0.7260	 accuracy = 0.5356
2025-02-17 11:08:07,325133062 | INFO | 569228 - epoch = 1 ---> loss = 0.6887	 accuracy = 0.5425
2025-02-17 11:08:08,325133307 | INFO | 569228 - epoch = 2 ---> loss = 0.6871	 accuracy = 0.5451
2025-02-17 11:08:08,325133547 | INFO | 569228 - epoch = 3 ---> loss = 0.6861	 accuracy = 0.5451
2025-02-17 11:08:08,325133774 | INFO | 569228 - epoch = 4 ---> loss = 0.6853	 accuracy = 0.5469
2025-02-17 11:08:08,325134005 | INFO | 569228 - epoch = 5 ---> loss = 0.6846	 accuracy = 0.5460
2025-02-17 11:08:09,325134303 | INFO | 569228 - epoch = 6 ---> loss = 0.6839	 accu

KeyboardInterrupt: 

In [ ]:

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

In [ ]:
forget_loader, _ = data_manager.get_loader_for('forget')
len(forget_loader)

66

In [13]:
train_loader, _ = data_manager.get_loader_for('train')

2025-02-17 18:47:52,352717800 | INFO | 739601 - TRAINING WITH 902 samples


In [19]:
forget = data_manager.partitions['forget']
len(forget)

242

In [14]:
for X,y in train_loader:
    print(X)
    print(y)

DataBatch(x=[61, 9], edge_index=[2, 120], edge_attr=[120, 3], batch=[61], ptr=[5])
tensor([-2, -1, -4,  0])
DataBatch(x=[52, 9], edge_index=[2, 118], edge_attr=[118, 3], batch=[52], ptr=[5])
tensor([-9, -2, -2, -4])
DataBatch(x=[27, 9], edge_index=[2, 50], edge_attr=[50, 3], batch=[27], ptr=[5])
tensor([-3, -4, -4, -2])
DataBatch(x=[61, 9], edge_index=[2, 122], edge_attr=[122, 3], batch=[61], ptr=[5])
tensor([-2, -3, -3, -3])
DataBatch(x=[25, 9], edge_index=[2, 44], edge_attr=[44, 3], batch=[25], ptr=[5])
tensor([-1, -1,  0,  0])
DataBatch(x=[49, 9], edge_index=[2, 102], edge_attr=[102, 3], batch=[49], ptr=[5])
tensor([ 0,  0, -3, -2])
DataBatch(x=[65, 9], edge_index=[2, 136], edge_attr=[136, 3], batch=[65], ptr=[5])
tensor([-4, -2, -4,  0])
DataBatch(x=[41, 9], edge_index=[2, 80], edge_attr=[80, 3], batch=[41], ptr=[5])
tensor([ 0,  0, -2, -1])
DataBatch(x=[41, 9], edge_index=[2, 82], edge_attr=[82, 3], batch=[41], ptr=[5])
tensor([-4, -3, -2,  0])
DataBatch(x=[38, 9], edge_index=[2, 

In [ ]:
from transformers import BertTokenizer

# Load the same tokenizer used for tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Iterate through your DataLoader
for X, y in forget_loader:
    decoded_text = tokenizer.batch_decode(X[0], skip_special_tokens=True)
    
    print(decoded_text)


['with easy operation you can make sweets happily you can learn how to make sweets with over 100 kinds of fun while playing there are many recipes including cakes cookies japanese sweets and so on', '[unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0]']
['there is no one in the world who could return from the dark side inviolate darkness and flame the dark side is an adventure game in the genre of hidden objects with plenty of minigames and puzzles unforgettable characters and complicated quests', '[unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0

In [ ]:
import matplotlib.pyplot as plt

# Parameters
num_images_to_display = 40
images_per_row = 4  # Number of images per row for a grid layout
images_displayed = 0

# Collect and display images
plt.figure(figsize=(15, 10))  # Set the figure size

for X, y in forget_loader:
    # Convert the batch to NumPy
    np_images = X.permute(0, 2, 3, 1).numpy()  # Change shape to (N, H, W, C)
    
    # Loop through images in the batch
    for img, label in zip(np_images, y):
        images_displayed += 1
        
        # Add subplot for each image
        plt.subplot(num_images_to_display // images_per_row, images_per_row, images_displayed)
        plt.imshow(img)  # Assuming the range is already suitable for visualization
        plt.title(f"Label: {label.item()}")  # Display label
        plt.axis("off")
        
        if images_displayed == num_images_to_display:
            break  # Stop after displaying the required number of images
    if images_displayed == num_images_to_display:
        break

plt.tight_layout()  # Adjust spacing between subplots
plt.savefig('2820_new_new.png')
plt.show()


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 3 is not equal to len(dims) = 4

<Figure size 1500x1000 with 0 Axes>

In [ ]:
loader, _ = data_manager.get_loader_for('train')

2025-01-31 17:09:43,-1121971173 | INFO | 346018 - TRAINING WITH 124594 samples


In [ ]:

from erasure.utils.config.global_ctx import bcolors


#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

2025-02-14 15:07:59,80324598 | INFO | 3335432 - Instantiating: erasure.model.gcn.DownstreamGCN


TypeError: __init__() missing 1 required positional argument: 'node_features'